# 基于 MindSpore 的 DDPM 扩散模型训练与推理示例

本 Notebook 使用 MindSpore 2.7.0 和 MindSpore NLP 0.5.1 实现一个完整的扩散模型训练与推理流程。

主要内容包括：
- 使用 `huggan/smithsonian_butterflies_subset` 蝴蝶图像数据集
- 实现 DDPM 噪声调度器
- 构建时间条件 U-Net 噪声预测网络
- 完成模型训练与从噪声生成图像的推理流程

运行环境假定：
- Python ≥ 3.9 且 < 3.12
- CANN ≥ 8.1.RC1（推荐 8.3.RC1）
- MindSpore ≥ 2.7.0
- MindSpore NLP == 0.5.1
- 设备为 Ascend，可通过环境变量控制


In [ ]:
import mindspore as ms
import mindtorch

# 【修改】将 device_target 改为 "CPU"
# 注意：CPU 模式通常不需要指定 device_id
# ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

# print("已切换至 CPU 模式")
# 目前使用测试模式PYNATIVE_MODE下的Ascend设备进行调试,后面可以改为GRAPH_MODE
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="Ascend", device_id=0)
print("已切换至 Ascend 模式，设备 ID: 0")
import contextlib
# 为旧版 mindtorch 补上 autograd.profiler，避免 zero_grad 中调用时报错
if not hasattr(mindtorch.autograd, "profiler"):
    class _DummyProfiler:
        @staticmethod
        @contextlib.contextmanager
        def record_function(name):
            yield
    mindtorch.autograd.profiler = _DummyProfiler()

/root/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/root/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/root/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/root/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
[WARNING] ME(3198:281472997273632,MainProcess):2025-12-15-23:05:54.274.000 [mindspore/context.py:1412] For 'context.set_context', the paramete

已切换至 Ascend 模式，设备 ID: 0


## 2. 配置训练参数

在开始构建数据管道和模型之前，先集中定义训练过程中需要用到的超参数：

- `image_size`：输入图像分辨率，这里使用 128×128；
- `train_batch_size` / `eval_batch_size`：训练与评估的 batch 大小；
- `num_epochs`：训练轮数；
- `learning_rate`：优化器的学习率；
- `save_image_epochs` / `save_model_epochs`：保存采样图片和模型权重的频率；
- `output_dir`：训练产物（权重、图片）的输出目录；
- `seed`：随机种子，保证实验可复现。

这些配置会在后续的数据预处理、模型构建、训练与推理各步骤中被统一引用，方便整体调整实验设置。

In [2]:
# 2. 配置训练参数

from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128  # 输入图像大小
    train_batch_size = 16
    eval_batch_size = 16  # 评估时的 batch size
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = 'fp16'  # MindSpore 中通过 amp_level 控制
    output_dir = 'ddpm-butterflies-128-ms'
    seed = 0

config = TrainingConfig()

## 3. 加载与处理数据集

本节加载 HuggingFace 上的 `huggan/smithsonian_butterflies_subset` 蝴蝶数据集，并使用 MindSpore 的 `GeneratorDataset` 封装为可迭代的数据管道：

1. 从 Hub 下载原始 PIL 图像；
2. 使用自定义的 `transform` 函数将图像 Resize 到目标分辨率、归一化到 [-1, 1] 并转换为 `CHW` 格式的 numpy 数组；
3. 通过 `ButterflyIterator` 将 HF 数据集包装成可索引的迭代器；
4. 使用 `GeneratorDataset` + `shuffle` + `batch` 得到训练用的 `data_loader`。

这一部分的输出是一个 MindSpore Dataset 对象，后续会在训练循环中按 batch 形式取出图像，并转换为 torch 张量交给模型。

In [3]:
# 3. 加载与处理数据集

import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
import mindspore as ms
import mindspore.dataset as ds
import mindspore.dataset.vision as vision
from datasets import load_dataset
from PIL import Image
import numpy as np
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128# 输入图像大小
    train_batch_size = 16
    eval_batch_size = 16  # 评估时的 batch size
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = 'fp16'# MindSpore 中通过 amp_level 控制
    output_dir = 'ddpm-butterflies-128-ms'
    seed = 0
    
config = TrainingConfig()    


# 1. 加载 HF 数据集
dataset = load_dataset("huggan/smithsonian_butterflies_subset", split="train")

# 2. 定义预处理操作 (使用 MindSpore Vision 算子)
def transform(data):
    # data 是一个字典，包含 'image' 键
    image = data['image']
    
    # 预处理流程：Resize -> RandomHorizontalFlip -> ToTensor -> Normalize
    # 注意：MindSpore 的 HWC -> CHW 转换通常在 ToTensor 或后续处理中
    # 这里为了简单，可以用 numpy/PIL 处理完直接转 Tensor
    image = image.resize((config.image_size, config.image_size))
    # ... 其他增强操作 ...
    
    # 归一化到 [-1, 1] 并转为 CHW 格式
    image = np.array(image) / 127.5 - 1.0
    image = image.transpose(2, 0, 1).astype(np.float32)
    return image

# 3. 封装为 MindSpore Dataset
class ButterflyIterator:
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        item = self.dataset[index]
        return transform(item),  # 返回 tuple

    def __len__(self):
        return len(self.dataset)

# 创建数据迭代器
data_loader = ds.GeneratorDataset(ButterflyIterator(dataset), column_names=["image"])
data_loader = data_loader.shuffle(buffer_size=1000)
data_loader = data_loader.batch(config.train_batch_size)

# 打印一下看看形状
for item in data_loader.create_dict_iterator():
    print(item['image'].shape)
    break

/root/.conda/envs/ms310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


mindtorch.Size([16, 3, 128, 128])


## 定义扩散模型

在此，我们搭建扩散模型。扩散模型是一类神经网络，其训练目标为从含噪输入中预测噪声程度略低的图像。在推理阶段，这类模型可通过迭代变换随机噪声来生成图像。

<p align="center">
    <img src="https://user-images.githubusercontent.com/10695622/174349667-04e9e485-793b-429a-affe-096e8199ad5b.png" width="800"/>
    <br>
    <em> 图片源自DDPM论文 (https://arxiv.org/abs/2006.11239). </em>
<p>

如果不熟悉其中的数学原理，不必过于担心。需要记住的核心要点是：我们的模型对应于公式中的概率分布 $p_{\theta}(x_{t-1}|x_{t})$ (换个通俗的说法就是：预测一张噪声程度略低的图像).

有意思的一点是，给图像添加噪声的操作其实非常简单，因此模型的训练可以按照如下步骤以半监督的方式进行：
1. 从训练数据集中选取一张图像。
2. 对该图像施加 $t$ 次随机噪声（这一步会得到上图中的 $x_{t-1}$ 和 $x_{t}$ ）
3. 将这张含噪图像与噪声步数 $t$ 一同输入至模型 
4. 基于模型的输出结果与含噪图像 $x_{t-1}$ 计算损失值。

随后，我们就可以采用梯度下降法，并重复上述流程多次以完成模型训练。

## 4. 定义扩散模型与噪声调度器

这一节中，我们使用 MindNLP 中封装的 diffusers 接口来构建整个扩散模型：

- `DDPMScheduler`：实现 DDPM 的噪声调度逻辑，负责前向加噪和反向去噪过程中的系数计算；
- `UNet2DModel`：时间条件 U-Net，用于在给定噪声图像和时间步 $t$ 的情况下预测噪声；
- `model.to("npu:0")`：将模型移动到 Ascend NPU 上进行加速训练。

模型结构（`block_out_channels`、`down_block_types`、`up_block_types`）与 HuggingFace 官方 `training_example.ipynb` 保持一致，
确保容量足以在 128×128 分辨率上学习蝴蝶图像分布。

大多数扩散模型都会采用 [U-net](https://arxiv.org/abs/1505.04597) 架构的某种变体，本文中我们也将使用这一架构。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/unet-model.png)

简而言之：
- 模型会让输入图像经过若干个 ResNet 层模块，每个模块将图像尺寸缩小一半；
- 随后图像再经过相同数量的模块，重新将其尺寸上采样恢复；
- 模型中设有跳跃连接（skip connections），将下采样路径上的特征层与上采样路径中对应的层连接起来。

该模型的一个核心特点是，其输出图像的尺寸与输入完全一致 —— 这正是我们此处所需的特性。
Diffusers 库为我们提供了便捷的 UNet2DModel 类，可在 PyTorch 中快速构建上述所需架构。

接下来，我们针对目标图像尺寸创建一个 U-net。

In [4]:
# 4. 定义模型与调度器

from mindnlp.diffusers import UNet2DModel, DDPMScheduler, DDPMPipeline

# 创建 Scheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

# 创建 UNet 模型
model = UNet2DModel(
    sample_size=config.image_size,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(128, 128, 256, 256, 512, 512),
    down_block_types=(
        "DownBlock2D", "DownBlock2D", "DownBlock2D", "DownBlock2D",
        "AttnDownBlock2D", "DownBlock2D"
    ),
    up_block_types=(
        "UpBlock2D", "AttnUpBlock2D", "UpBlock2D", "UpBlock2D",
        "UpBlock2D", "UpBlock2D"
    )
)
model.to("npu:0") 

print("模型已成功移动到 NPU")
# 打印模型结构确认
print(model)

Modular Diffusers is currently an experimental feature under active development. The API is subject to breaking changes in future releases.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
模型已成功移动到 NPU
UNet2DModel(
  (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=128, out_features=512, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=512, out_features=512, bias=True)
  )
  (down_blocks): ModuleList(
    (0-1): 2 x DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 128, eps=1e-05, affine=True)
          (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=512, out_features=128, bias=True)
          (norm2): GroupNorm(32, 128, eps=1e-05, affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (nonlinearity): SiLU()
        )
      )
 

In [5]:
# 打印一下看看参数是否正常
params = model.trainable_params()
print(f"参数数量: {len(params)}")
print(f"第一个参数的类型: {type(params[0])}") 
# 这里应该输出 <class 'mindspore.common.parameter.Parameter'>

参数数量: 450
第一个参数的类型: <class 'torch.Tensor'>


## 5. 定义损失函数与优化器

和原始 PyTorch 示例一样，我们在这里选择：

- `MSELoss`：让模型在所有时间步上预测的噪声尽可能接近真实噪声；
- `AdamW`：带权重衰减的 Adam 优化器，适合训练 U-Net 这类较深的网络；
- `device`：通过 `torch.device("npu:0")` 将模型与张量统一放在 Ascend NPU 上，由 MindNLP/mindtorch 做后端调度。

之后的训练步骤会基于这套损失与优化器进行标准的 `loss.backward()` + `optimizer.step()` 迭代。

In [ ]:
# 5. 定义损失函数与优化器（由 mindnlp/mindhf 代理到 MindSpore）
import torch
import torch.nn as nn

device = torch.device("npu:0" if hasattr(torch, "npu") else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
loss_fn = nn.MSELoss()


In [7]:
# 6. 编写训练流程 (Forward & Backward) 

def train_step(clean_images: torch.Tensor):
    model.train()
    optimizer.zero_grad()

    bs = clean_images.shape[0]
    noise = torch.randn_like(clean_images)

    # 使用 scheduler 的 config.num_train_timesteps
    timesteps = torch.randint(
        0,
        noise_scheduler.config.num_train_timesteps,
        (bs,),
        device=device,
        dtype=torch.long,
    )

    noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)
    noise_pred = model(noisy_images, timesteps).sample
    loss = loss_fn(noise_pred, noise)

    loss.backward()
    optimizer.step()

    return loss.detach()

在这一步，我们实现一个单步的训练函数 `train_step`：

1. 从干净图像 `clean_images` 中采样同形状的高斯噪声 `noise`；
2. 从调度器的时间步范围中随机采样一个整型向量 `timesteps`；
3. 调用 `noise_scheduler.add_noise` 得到噪声图像 `noisy_images`；
4. 通过 U-Net 模型预测噪声 `noise_pred = model(noisy_images, timesteps).sample`；
5. 使用 `MSELoss` 计算预测噪声与真实噪声之间的距离；
6. 调用 `loss.backward()` 和 `optimizer.step()` 完成一次参数更新。


In [ ]:
# 7. 评估与采样辅助函数
from PIL import Image

def make_grid(images, rows, cols):
    w, h = images[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i % cols * w, i // cols * h))
    return grid


def evaluate(config, epoch, pipeline):
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.Generator(device=device).manual_seed(config.seed),
    ).images

    image_grid = make_grid(images, rows=4, cols=4)
    samples_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(samples_dir, exist_ok=True)
    image_grid.save(os.path.join(samples_dir, f"epoch_{epoch:04d}.png"))


#### 8. 推理与评估

实现从纯噪声生成图像的过程。

In [9]:
# 8. 启动训练循环并在训练过程中保存模型与采样图片

from tqdm import tqdm

model.set_train(True)

for epoch in range(config.num_epochs):
    step_loss = []
    with tqdm(total=data_loader.get_dataset_size()) as progress_bar:
        progress_bar.set_description(f"Epoch {epoch}")
        
        for batch in data_loader.create_dict_iterator():
            # 从 MindSpore Dataset 中取出的数据先转为 numpy，再转为 torch.Tensor
            images_np = batch['image']
            images_np = np.array(images_np)
            clean_images = torch.tensor(images_np, dtype=torch.float32, device=device)

            # 执行一步训练（纯 torch 接口，由 mindnlp/mindhf 代理到 MindSpore/Ascend）
            loss = train_step(clean_images)
            step_loss.append(loss.cpu().item())
            
            progress_bar.update(1)
            progress_bar.set_postfix(loss=loss.cpu().item())
    
    avg_loss = float(np.mean(step_loss))
    print(f"Epoch {epoch} finished. Avg Loss: {avg_loss:.4f}")

    # 周期性保存采样图片（从纯噪声生成），参考 training_example.ipynb
    if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
        pipeline = DDPMPipeline(unet=model, scheduler=noise_scheduler)
        evaluate(config, epoch + 1, pipeline)

    # 周期性保存模型权重
    if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
        os.makedirs(config.output_dir, exist_ok=True)
        ckpt_path = os.path.join(config.output_dir, "unet_ddpm_mindnlp_hf.pt")
        torch.save(model.state_dict(), ckpt_path)
        print(f"Saved checkpoint to {ckpt_path}")

Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:15<00:00,  4.18it/s, loss=0.365]


Epoch 0 finished. Avg Loss: 0.3609


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:14<00:00,  4.49it/s, loss=0.404]


Epoch 1 finished. Avg Loss: 0.3603


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:14<00:00,  4.48it/s, loss=0.344]


...

Epoch 48 finished. Avg Loss: 0.3592


Epoch 49: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:14<00:00,  4.38it/s, loss=0.351]


Epoch 49 finished. Avg Loss: 0.3608


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:03<00:00, 15.80it/s]


Saved checkpoint to ddpm-butterflies-128-ms/unet_ddpm_mindnlp_hf.pt


训练循环整体流程：

1. 遍历 `num_epochs` 个训练轮次；
2. 每个 epoch 中从 `data_loader` 取出一个 batch，转为 NPU 上的 torch 张量；
3. 调用 `train_step` 完成一次前向 + 反向 + 参数更新；
4. 使用 `tqdm` 展示训练进度和当前 loss；
5. 周期性使用 `DDPMPipeline` 从纯噪声采样，保存训练过程中的生成图片快照；
6. 周期性保存当前模型权重，便于后续单独加载做推理。

这样可以方便地观察 loss 曲线以及生成图像质量的演化趋势，同时保留中间检查点用于调试和复现。

## 9. 推理与评估：从纯噪声生成图像

在训练完成后，可以单独加载最新的模型权重，从纯噪声出发执行 DDPM 反向采样，
生成一组蝴蝶图像。这里同样使用 `DDPMPipeline` 进行推理。

In [10]:
# 9. 推理：从纯噪声采样最终图像网格

# 重新构建模型并加载最新权重（确保推理和训练结构一致）
inference_model = UNet2DModel(
    sample_size=config.image_size,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(128, 128, 256, 256, 512, 512),
    down_block_types=(
        "DownBlock2D", "DownBlock2D", "DownBlock2D", "DownBlock2D",
        "AttnDownBlock2D", "DownBlock2D"
    ),
    up_block_types=(
        "UpBlock2D", "AttnUpBlock2D", "UpBlock2D", "UpBlock2D",
        "UpBlock2D", "UpBlock2D"
    )
).to(device)

ckpt_path = os.path.join(config.output_dir, "unet_ddpm_mindnlp_hf.pt")
state_dict = torch.load(ckpt_path, map_location=device)
inference_model.load_state_dict(state_dict)
inference_model.eval()

# 使用新的调度器与 pipeline 做推理
inference_scheduler = DDPMScheduler(num_train_timesteps=1000)
inference_pipeline = DDPMPipeline(unet=inference_model, scheduler=inference_scheduler)

images = inference_pipeline(
    batch_size=config.eval_batch_size,
    generator=torch.Generator(device=device).manual_seed(config.seed),
).images

final_grid = make_grid(images, rows=4, cols=4)
final_path = os.path.join(config.output_dir, "final_samples_grid.png")
os.makedirs(config.output_dir, exist_ok=True)
final_grid.save(final_path)
final_path


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:59<00:00, 16.86it/s]


'ddpm-butterflies-128-ms/final_samples_grid.png'